# 1. 콜백을 사용하여 모델 훈련 과정 제어하기

- 콜백은 모델의 fit() 메서드가 호출될 떄 전달되는 객체(특정 메서드를 구현한 클래스 객체)입니다. 훈련하는 동안 모델을 여러 지점에서 호출합니다. 콜백은 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재, 모델 상태 변경을 처리할 수 있다.

## 1. 콜백 예시

- 모델 체크포인트 저장: 훈련하는 동안 어떤 지점에서 모델 현재 가중치를 저장한다.
- 조기 종료: 검증 손실이 더 이상 향상되지 않을 때 훈련을 중지한다.
- 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정한다.
- 훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이트 될 때마다 시각화합니다.

### ModelCheckpoint와 EarlyStopping 콜백

- EarlyStopping 콜백: 정해진 에포크 동안 모니터링 지표가 향상되지 않음
- ModelCheckpoint: 훈련 동안 모델을 저장

In [1]:
import keras

Using TensorFlow backend.


In [3]:
callback_list = [
    keras.callbacks.EarlyStopping( # 성능 향상이 멈추면 훈련을 중지한다.
        monitor='val_acc',
        patience=1
    ),
    keras.callbacks.ModelCheckpoint( # 에폭마다 현재 가중치를 젖아한다.
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

In [4]:
# fit, fit_generator에 callback을 부를 수 있다.
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# model.fit(x, y, epochs=10, batch_size=32, callbacks=callback_list, validation_data=(x_val, y_val))

NameError: name 'model' is not defined

### ReduceLROnPlateau 콜백

- 검증 손실이 향상되지 않을 떄 학습률을 작게 할 수 있다.

In [5]:
callback_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1, # 콜백이 호출될 떄 학습률을 10배로 줄인다.
        patience=10
    )
]

# model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

### 자신만의 콜백 만들기

- on_epoch_begin: 각 에포크가 시작할 때 호출한다.
- on_epoch_end: 각 에포크가 끝날 떄 호출한다.

- on_batch_begin: 각 배치가 시작되기 전에 호출한다.
- on_batch_end: 각 배치가 끝난 후에 호출한다.

- on_train_begin: 훈련이 시작될 때 호출한다.
- on_train_end: 훈련이 끝날 때 호출한다.

In [7]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs) # 각 층의 활성화 출력을 반환하는 Model 객체
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError("Requires validation data.")
            
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.perdict(validation_sample)
        f = open('activations_at_epoch' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()

# 2. 텐서보드: 텐서플로의 시각화 프레임워크

- 텐서보드의 핵심 목적은 훈련 모델의 내부에서 일어나는 모든 것을 시각적으로 모니터링할 수 있도록 돕는 것이다. 모델의 최종 손실 외에 더 많은 정보를 모니터링하면 모델 작동에 대한 명확한 그림을 그릴 수 잇습니다. 결국 모델을 더 빠르게 개선할 수 있습니다. 텐서보드는 여러가지 멋진 기능을 제공합니다. 모두 브라우저에서 작동합니다.

In [1]:
import keras 
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data('imdb.npz', num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

Using TensorFlow backend.


In [2]:
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [4]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir',
        histogram_freq=1, # 1 에포크마다 활성화 출력 히스토그램 기록
        embeddings_freq=1 # 1 에포크마다 임베딩 데이터 기록
    )
]
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 24s 1ms/step - loss: 0.5695 - acc: 0.7108 - val_loss: 0.4169 - val_acc: 0.8404
Epoch 2/20
20000/20000 [==============================] - 20s 1ms/step - loss: 0.3752 - acc: 0.8148 - val_loss: 0.5406 - val_acc: 0.7622
Epoch 3/20
20000/20000 [==============================] - 21s 1ms/step - loss: 0.2806 - acc: 0.7295 - val_loss: 0.5016 - val_acc: 0.7372
Epoch 4/20
20000/20000 [==============================] - 20s 1ms/step - loss: 0.2028 - acc: 0.6701 - val_loss: 0.8359 - val_acc: 0.5434
Epoch 5/20
20000/20000 [==============================] - 20s 1ms/step - loss: 0.1562 - acc: 0.5789 - val_loss: 0.7343 - val_acc: 0.5138
Epoch 6/20
20000/20000 [==============================] - 20s 1ms/step - loss: 0.1227 - acc: 0.4583 - val_loss: 0.8403 - val_acc: 0.4276
Epoch 7/20
20000/20000 [==============================] - 20s 1ms/step - loss: 0.1056 - acc: 0.3709 - val_loss: 0.9710 - val_acc:

In [6]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

# 3. 정리

- 케라스 콜백은 훈련하는 동안 모델을 모니터링하고 모델 상태를 바탕으로 자동으로 작업을 수행하는 손쉬운 방법입니다.
- 텐서플로를 사용하면 텐서보드를 이용하여 모델 상황을 브라우저에서 시각화할 수 있습니다. 케라스 모델에서는 Tensorboard 콜백을 통해 사용합니다.